In [4]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
import os # Importar a biblioteca 'os' para manipulação de caminhos

# --- Etapa 0: Montar o Google Drive ---
try:
    drive.mount('/content/drive', force_remount=False)
    print("Google Drive montado com sucesso.")
except Exception as e:
    print(f"Erro ao montar o Google Drive: {e}")
    raise e

# --- Etapa 1: Definir caminhos e Carregar os TRÊS arquivos ---

path_daiane = '/content/drive/MyDrive/pesquisa_mestrado_2025/tabela_dados_experimentais_daiane_francisca.xlsx'
path_victor = '/content/drive/MyDrive/pesquisa_mestrado_2025/tabela_dados_experimentais_victor_cardoso.xlsx' # Corrigido se houver erro de digitação
path_processado = '/content/drive/MyDrive/pesquisa_mestrado_2025/resultados estruturados/Tabela 1_ Identificação da Amostra e Pré-processamento.xlsx'

# *** NOVA ETAPA: Definir a pasta de destino para os resultados ***
# Vamos extrair o diretório (pasta) do arquivo 'path_processado'
output_folder_path = os.path.dirname(path_processado)
print(f"\nOs arquivos de resultado CSV serão salvos em: {output_folder_path}")

print("\nCarregando arquivos...")
try:
    df_daiane = pd.read_excel(path_daiane)
    print(f"Arquivo 'df_daiane' carregado ({len(df_daiane)} linhas).")

    df_victor = pd.read_excel(path_victor)
    print(f"Arquivo 'df_victor' carregado ({len(df_victor)} linhas).")

    df_processado = pd.read_excel(path_processado)
    print(f"Arquivo 'df_processado' (seus resultados) carregado ({len(df_processado)} linhas).")

except FileNotFoundError as e:
    print(f"\nERRO CRÍTICO: Arquivo não encontrado.")
    print(f"Detalhes: {e}")
    raise e
except Exception as e:
    print(f"\nERRO CRÍTICO ao ler arquivos: {e}")
    raise e

# --- Etapa 2: Limpar df_processado E CORRIGIR UNIDADE ---

col_porosidade_processada_origem = 'Porosidade Global Média (%)'
if col_porosidade_processada_origem in df_processado.columns:
    df_processado['Porosidade_Processada'] = df_processado[col_porosidade_processada_origem].astype(str).str.replace('%', '', regex=False).astype(float)

    if df_processado['Porosidade_Processada'].mean() < 1.0:
        print("\nDetectado que 'Porosidade_Processada' está como fração (ex: 0.2725).")
        print("Convertendo para porcentagem (multiplicando por 100) para equiparar às referências.")
        df_processado['Porosidade_Processada'] = df_processado['Porosidade_Processada'] * 100
    else:
        print(f"\nValores de 'Porosidade_Processada' parecem já estar em formato de porcentagem (ex: {df_processado['Porosidade_Processada'].mean():.2f}). Nenhuma conversão (x100) será feita.")

else:
    print(f"\nERRO: Coluna '{col_porosidade_processada_origem}' não encontrada em df_processado.")
    raise ValueError(f"Coluna '{col_porosidade_processada_origem}' não encontrada.")


# --- Etapa 3: Criar Chave Comum em df_processado ---

keys_daiane = df_daiane['Colunas ID'].dropna().unique()
keys_victor = df_victor['Classificação de Abrão'].dropna().unique()
all_keys = np.concatenate([keys_daiane, keys_victor])

def find_key_in_string(long_filename_str, key_list):
    for key in key_list:
        if key in str(long_filename_str):
            return key
    return None

col_id_processado = 'ID da Amostra (Nome do Arquivo Base)'
if col_id_processado in df_processado.columns:
    df_processado['ID_Comum'] = df_processado[col_id_processado].apply(
        lambda x: find_key_in_string(x, all_keys)
    )
    print("Coluna 'ID_Comum' criada em df_processado para matching.")
else:
    print(f"\nERRO: Coluna '{col_id_processado}' não encontrada em df_processado.")
    raise ValueError(f"Coluna '{col_id_processado}' não encontrada.")


# --- Etapa 4 & 5: Comparação com Daiane (Merge e Cálculo de Erro) ---

print("\n" + "="*50)
print("Comparação com 'df_daiane'")
print("="*50)

df_comp_daiane = pd.merge(
    df_processado,
    df_daiane,
    left_on='ID_Comum',
    right_on='Colunas ID',
    how='inner'
)

if df_comp_daiane.empty:
    print("Nenhuma amostra comum encontrada entre 'df_processado' e 'df_daiane'.")
else:
    print(f"Total de {len(df_comp_daiane)} amostras comuns encontradas com Daiane.")

    cols_metodos_daiane = ['Porosimetria direta', 'Gravimetria', 'Porosimetria indireta', 'Porosimetria a gás', 'μTC-RX']
    cols_calculadas_daiane = []

    for col_metodo in cols_metodos_daiane:
        if col_metodo in df_comp_daiane.columns:
            nome_col_erro = f'Erro_vs_{col_metodo}'
            df_comp_daiane[nome_col_erro] = df_comp_daiane['Porosidade_Processada'] - df_comp_daiane[col_metodo]
            cols_calculadas_daiane.append(nome_col_erro)

            nome_col_erro_rel = f'ErroRelativo_vs_{col_metodo}'
            df_comp_daiane[nome_col_erro_rel] = (df_comp_daiane['Porosidade_Processada'] - df_comp_daiane[col_metodo]) / df_comp_daiane[col_metodo]
            df_comp_daiane[nome_col_erro_rel] = df_comp_daiane[nome_col_erro_rel].replace([np.inf, -np.inf], np.nan)
            cols_calculadas_daiane.append(nome_col_erro_rel)

    print("\nResultados da Comparação (Daiane):")
    cols_exibir_daiane = ['ID_Comum', 'Porosidade_Processada'] + cols_metodos_daiane + cols_calculadas_daiane
    cols_exibir_daiane = [c for c in cols_exibir_daiane if c in df_comp_daiane.columns]

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(df_comp_daiane[cols_exibir_daiane])

    # *** CAMINHO DE SALVAMENTO ATUALIZADO ***
    path_csv_daiane = os.path.join(output_folder_path, 'comparacao_resultados_vs_daiane_ERRO_CORRIGIDO.csv')
    df_comp_daiane.to_csv(path_csv_daiane, index=False, decimal=',', sep=';')
    print(f"\nResultados da comparação com Daiane salvos em: {path_csv_daiane}")


# --- Etapa 6 & 7: Comparação com Victor (Merge e Cálculo de Erro) ---

print("\n" + "="*50)
print("Comparação com 'df_victor'")
print("="*50)

df_comp_victor = pd.merge(
    df_processado,
    df_victor,
    left_on='ID_Comum',
    right_on='Classificação de Abrão',
    how='inner'
)

if df_comp_victor.empty:
    print("Nenhuma amostra comum encontrada entre 'df_processado' e 'df_victor'.")
else:
    print(f"Total de {len(df_comp_victor)} amostras comuns encontradas com Victor.")

    cols_metodos_victor = [
        'Porosidade Obtida por Victor pela segmentação proposta por victor',
        'Otsu por Victor pela segmentação proposta',
        'Watershed por Victor pela segmentação proposta',
        'K-means por Victor pela segmentação proposta',
        'PVE por Victor pela segmentação proposta',
        'Porosidade pelo porosimetro de gás helio - abrao'
    ]
    cols_calculadas_victor = []

    for col_metodo in cols_metodos_victor:
        if col_metodo in df_comp_victor.columns:
            nome_curto = col_metodo.split(' ')[0]
            if 'gás helio' in col_metodo: nome_curto = 'GasHelioAbrao'

            nome_col_erro = f'Erro_vs_{nome_curto}_Victor'
            df_comp_victor[nome_col_erro] = df_comp_victor['Porosidade_Processada'] - df_comp_victor[col_metodo]
            cols_calculadas_victor.append(nome_col_erro)

            nome_col_erro_rel = f'ErroRelativo_vs_{nome_curto}_Victor'
            df_comp_victor[nome_col_erro_rel] = (df_comp_victor['Porosidade_Processada'] - df_comp_victor[col_metodo]) / df_comp_victor[col_metodo]
            df_comp_victor[nome_col_erro_rel] = df_comp_victor[nome_col_erro_rel].replace([np.inf, -np.inf], np.nan)
            cols_calculadas_victor.append(nome_col_erro_rel)

    print("\nResultados da Comparação (Victor):")
    print("(Valores 'NaN' indicam dados ausentes no arquivo de Victor)")

    cols_exibir_victor = ['ID_Comum', 'Porosidade_Processada'] + cols_metodos_victor + cols_calculadas_victor
    cols_exibir_victor = [c for c in cols_exibir_victor if c in df_comp_victor.columns]

    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(df_comp_victor[cols_exibir_victor].set_index('ID_Comum').T)

    # *** CAMINHO DE SALVAMENTO ATUALIZADO ***
    path_csv_victor = os.path.join(output_folder_path, 'comparacao_resultados_vs_victor_ERRO_CORRIGIDO.csv')
    df_comp_victor.to_csv(path_csv_victor, index=False, decimal=',', sep=';')
    print(f"\nResultados da comparação com Victor salvos em: {path_csv_victor}")

print("\n" + "="*50)
print("Análise de Erro Concluída. Arquivos salvos na pasta de origem.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado com sucesso.

Os arquivos de resultado CSV serão salvos em: /content/drive/MyDrive/pesquisa_mestrado_2025/resultados estruturados

Carregando arquivos...
Arquivo 'df_daiane' carregado (9 linhas).
Arquivo 'df_victor' carregado (9 linhas).
Arquivo 'df_processado' (seus resultados) carregado (24 linhas).

Detectado que 'Porosidade_Processada' está como fração (ex: 0.2725).
Convertendo para porcentagem (multiplicando por 100) para equiparar às referências.
Coluna 'ID_Comum' criada em df_processado para matching.

Comparação com 'df_daiane'
Total de 4 amostras comuns encontradas com Daiane.

Resultados da Comparação (Daiane):
  ID_Comum  Porosidade_Processada  Porosimetria direta  Gravimetria  Porosimetria indireta  Porosimetria a gás  μTC-RX  Erro_vs_Porosimetria direta  ErroRelativo_vs_Porosimetria direta  Erro_vs_Gravimetria  ErroRelativo_v